In [ ]:
#install dependencies
%pip install requests pandas 
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


In [135]:
#get data 
%cd F1 dataset 
circuits = pd.read_csv('circuits.csv')
drivers = pd.read_csv('drivers.csv')
lap_times = pd.read_csv('lap_times.csv')
pit_stops = pd.read_csv('pit_stops.csv') 
qualifying = pd.read_csv('qualifying.csv')
results = pd.read_csv('results.csv')
seasons = pd.read_csv('seasons.csv')
status = pd.read_csv('status.csv')

#Explore the datasets
datasets = {
    "Circuits": circuits,
    "Drivers": drivers,
    "Lap Times": lap_times,
    "Pit Stops": pit_stops,
    "Qualifying": qualifying,
    "Results": results,
    "Seasons": seasons,
    "Status": status
}

[Errno 2] No such file or directory: 'F1 dataset'
/home/penguin/Documents/Code/F1RaceOutcomePrediction/F1 dataset


/home/penguin/Documents/Code/F1RaceOutcomePrediction/venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [152]:
#Data
lap_times = datasets['Lap Times']
drivers = datasets['Drivers']
circuits = datasets['Circuits']


#Data merging
merged_data = lap_times.merge(drivers, on='driverId', how='inner')
merged_data = merged_data.merge(circuits, left_on='driverId', right_on='circuitId', how='inner')

#Convert lap times to seconds
import pandas as pd

def convert_to_seconds(time_str):
    if isinstance(time_str, str):
        minutes, seconds = time_str.split(':')
        return int(minutes) * 60 + float(seconds)
    return time_str

# Ensure the correct column name is used
if 'milliseconds' in merged_data.columns:
    merged_data['lapTime'] = merged_data['milliseconds'].apply(lambda x: x / 1000)
else:
    raise KeyError("Column 'milliseconds' not found in the merged dataframe")


In [156]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

#features
categorical_features = ['driverId', 'circuitId']  
one_hot = OneHotEncoder(sparse_output=False)  

#transformer
transformer = ColumnTransformer(
    transformers=[
        ('one_hot', one_hot, categorical_features)
    ],
    remainder='passthrough'  #
)
encoded_data = transformer.fit_transform(merged_data)

In [158]:
import torch
import torch.nn as nn
import torch.optim as optim

#define model
class LapTimePredictor(nn.Module):
    def __init__(self, input_size):
        super(LapTimePredictor, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(input_size, 64)  
        self.fc2 = nn.Linear(64, 32)        
        self.fc3 = nn.Linear(32, 1)          

    def forward(self, x):
        x = torch.relu(self.fc1(x))  
        x = torch.relu(self.fc2(x))  
        x = self.fc3(x)            
        return x

#initialize model 
input_size = X_train.shape[1] # Number of features
model = LapTimePredictor(input_size)

#loss function
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)